In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Modelling
- first hour
    - experimentral alg
    - pipeline
    - examples
    - homework
- second hour
    - pokemon
    - intro I.P

ゴール： 現実的な問題をコンピュータの能力内で解く  
JEA SEAなどのコミュニティサイトがある.  
論文をまとめてあるサイト？gatech?なども使える．  
アルゴリズムを論文に乗せるのに大事なこと，実験に大事なこと？  
コードを見せることではなく，問題がなんなのか，その解法の長所と短所はなんなのか，にたいする洞察を示すことが大事である．  

# EA pipeline Step  
<br>
<div style="text-align:center">problem(client) -> formalization -> algorithm design -> implementation -> experimentation</div>
<br>
この一連のステップそれぞれからPractical Insightを導き，さらにFeedbackを付け加える  
homeworkはexperimentationの辺りから何か出される

## Formalization
needs storong mathematical foundations  
理論上重要なことが実践では重要ではないことがある．  
Bias filtering vs Approximation algorithm

## Algorithm design
どんな点でBest solutionなのか?(最適解を出せるのか，早いのか)  
実装時に発生する問題を考慮してデザインすることが大事

## Implementation
Programming language, team, task distribution...  
専門知識なしで，誰が実装しても良いようにしたい．  
実装時間 vs 実行時間

## Experiments
実験の設計(何をどうやって測定するのか)を定める  
実験時の「バイアス」に注意する  
- 認証バイアス(confirmation bias)世の中で正しいとされているもの，正しいと自分が信じているものを正しいとしてしまうこと
- 文化バイアス(culture bias)メディアなど？

## Insight
- 洞察の種類(問題，アプローチ)
    - 結果の解釈
    - 仮説検証
- SOFT skills

# Practical Examples
- road network matching
- medical image analysis
- 3D point set residues

## Road network matching
地図の一部がどこのものであるかを調べる  
交差点と交差点への彩色問題によって，$O(n^4 n'^{3} \log n')$で解ける. YAGO先生の論文より

## Medical Image Analysis

## 3D Point Cloud Matching
複数の角度から撮ったうさぎの3Dモデルができるって話  
最低3つの同じ位置にある点を見つける必要がある．  $O(n^6)$  
最新では$O(n^2)$くらい？
最悪計算量のオーダが多くても，現実のデータで例えば全てのなんらかの対象となるピクセルがある領域に固まっていることはないなどの仮定を置けば有用なアルゴリズムとなる．

# Toy Problem
which is the strongest Pokemon?
Formalization
- how many pokemon are there?
    251? FIRST GENERATION Pokemons.
- what does "strong" mean?
    winners of Combats
- read bibliograohy
    - 参考：https://hubpages.com/entertainment/Top-10-Strongest-First-Generation-Pokemons

Algorithm design
- combat system はユーザのコマンドを必要とする
- タイプシステム
- Moves can be developed using points during the game
- Official codeを入手することはできない

## Implementation
easy to implement  
no money  
python  

Home work
1. review the python code, feel free to improve it.
2. design experiment(s) to find out the stronger pokemon and pokemon team
    team: 6 pokemon, 1匹ずつ戦い，勝ったほうが次のポケモンと戦う
3. Report
    - results/comments insight!
    - Given the restrictions, how would you improve the process(Formalization, design, implementation)
    - restrictionsがなければどうやってimproveするか？
    - 結果は予想どうりのものか？そうでなければ何が問題だったのか？

とりあえず思いつくのは総当たり戦をして，勝った回数が一番多いやつが強い  
より早くしたいならランダムリーグ戦をやらせる．これは近似解になる．(タイプシステムがあるので最強とは限らない)

In [2]:
from lesson3Code.pokemonTournament.pokemonDef import Monster
import lesson3Code.pokemonTournament.pokemonFunctions as pf
import sys, random

listOfNamesAndTypes=pf.readPokemonTypesFromFile("lesson3Code/pokemonTournament/listOfPokemon1.txt")
pokeList=pf.readPokemonListFromFile("lesson3Code/pokemonTournament/listOfPokemon2.txt")

pokedex={}
for monster in pokeList:
    pokedex[monster.name]=monster

for name,listOfTypes in listOfNamesAndTypes:
    for t in listOfTypes:
        pokedex[name].variety.append(t)

In [3]:
def RoundRobinExperiment(k=1, battle=pf.battle, result=True):
    win_times = np.zeros(len(pokeList), dtype='i')
    for i in range(k):
        for player in range(len(pokeList)):
            win_time = 0
            for enemy in range(len(pokeList)):
                if player == enemy: continue
                winner = battle(pokeList[player], pokeList[enemy])
                if winner == 0: win_time += 1
            win_times[player] += win_time

    if result:
        strongest = win_times.argmax()
        print("The strongest pokemon is", pokeList[strongest].name, "id: ", strongest)
        print("The strongest pokemon parameter", pokeList[strongest])
        print("Strongest Pokemon Top 10")
        for i in win_times.argsort()[::-1][:10]:
            print(pokeList[i])
    
    return win_times

総当たり: $O(n^2)$  
$K$回の総当たり: $O(Kn^2)$  
この場合は，「あるポケモンが自分より弱いポケモンが何種類いるか」が強さの基準になっている．  
- 長所
    - ポケモンの能力値がわからなくても強さを測定できる．
    - 実装が簡単である
- 短所
    - 時間がかかる
    - Combat Systemに乱数(一様分布)が利用されているため,実験の度に結果が変わる
        - 何回繰り返したところで最強とは限らない

仮説
- 同じタイプの場合，攻撃力，防御力，HPで力関係の期待値がわかる
    - 毎ターンそれぞれが対決
    - 攻撃力と防御力の変動値は同じタイプであれば一様分布で決まる

In [7]:
from lesson3Code.pokemonTournament.pokemonDef import Monster,superEffective,notEffective,noEffect

def SpeedBattle(p1, p2, resetHP=True):
    effectiveness = 0.3
    max_rounds = 1000
    
    if resetHP:
        pf.ResetHP(p1)
        pf.ResetHP(p2)
    
    if noEffect(p1, p2) and noEffect(p2, p1): return 2
    elif noEffect(p1, p2): return 1
    elif noEffect(p2, p1): return 0
    
    attackCenter1=1
    if superEffective(p1, p2): attackCenter1+=effectiveness
    elif notEffective(p1, p2): attackCenter1-=effectiveness

    attackCenter2=1
    if superEffective(p2, p1): attackCenter2+=effectiveness
    elif notEffective(p2, p1): attackCenter2-=effectiveness
    
    get_damage1 = max(0, (round(p2.attack * attackCenter2) - p1.defense * 0.9))
    survive_rounds1 = p1.hitpoint[1] / get_damage1 if get_damage1 > 0 else max_rounds
        
    get_damage2 = max(0, (round(p1.attack * attackCenter1) - p2.defense * 0.9))
    survive_rounds2 = p2.hitpoint[1] / get_damage2 if get_damage2 > 0 else max_rounds
        
    if survive_rounds1 == survive_rounds2: 
        if p1.speed == p2.speed: return 0
        return 0 if p1.speed > p2.speed else 1
    
    return 0 if survive_rounds1 > survive_rounds2 else 1

In [8]:
K = 1
normal_result = RoundRobinExperiment(K)
print("====================== Speed Battle =========================")
speed_result = RoundRobinExperiment(battle=SpeedBattle)

The strongest pokemon is Cloyster id:  90
The strongest pokemon parameter Monster(name='Cloyster', index=90, attack=95, defense=180, speed=85, hitpoint=[50, 50], variety=['Water', 'Ice'])
Strongest Pokemon Top 10
Monster(name='Cloyster', index=90, attack=95, defense=180, speed=85, hitpoint=[50, 50], variety=['Water', 'Ice'])
Monster(name='Gyarados', index=129, attack=125, defense=79, speed=100, hitpoint=[95, 52], variety=['Water', 'Flying'])
Monster(name='Kingler', index=98, attack=130, defense=115, speed=50, hitpoint=[55, 47], variety=['Water'])
Monster(name='Mewtwo', index=149, attack=110, defense=90, speed=154, hitpoint=[106, 35], variety=['Psychic'])
Monster(name='Dragonite', index=148, attack=134, defense=95, speed=100, hitpoint=[91, 35], variety=['Dragon', 'Flying'])
Monster(name='Mew', index=150, attack=100, defense=100, speed=100, hitpoint=[100, -15], variety=['Psychic'])
Monster(name='Kabutops', index=140, attack=115, defense=105, speed=70, hitpoint=[60, -7], variety=['Rock', 

In [9]:
normal_result.argsort() == speed_result.argsort()

array([ True, False, False, False, False,  True, False,  True, False,
       False, False,  True, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False,  True,
       False, False, False, False,  True, False, False, False, False,
       False, False, False, False, False,  True, False, False, False,
       False, False, False, False, False, False, False, False, False,
        True, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False,  True, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False,  True, False, False, False, False,
       False, False,

順位が同じ，という基準では少し厳しい気がするので，レーベンシュタイン距離を基準にする

In [53]:
# Levenstein Distance by Dynamic Programming
def LevensteinDistance(l1, l2):
    len1 = len(l1) + 1
    len2 = len(l2) + 1
    
    dp = np.zeros((len1, len2), dtype='i')
    for row in range(len1):
        for col in range(len2):
            if row == 0: 
                dp[row][col] = col
                continue
                
            if col == 0:
                dp[row][col] = row
                continue
            
            c = 0 if l1[row-1] == l2[col-1] else 1
            dp[row][col] = min(dp[row-1][col]+1, dp[row][col-1]+1, dp[row-1][col-1] + c)

    print(dp)
    
    return dp[-1][-1]

In [54]:
l1 = np.random.randint(1,10,5)
l2 = np.random.randint(1,10,4)
print(l1)
print(l2)
LevensteinDistance(l1,l2)

[8 1 6 8 9]
[3 7 8 2]
[[0 1 2 3 4]
 [1 1 2 2 3]
 [2 2 2 3 3]
 [3 3 3 3 4]
 [4 4 4 3 4]
 [5 5 5 4 4]]


4

In [10]:
import time

startNormal = time.time()
RoundRobinExperiment(result=False)
endNormal = time.time()
print("Normal Battle Elapsed Time:", endNormal - startNormal)

startSpeed = time.time()
RoundRobinExperiment(battle=SpeedBattle, result=False)
endSpeed = time.time()
print("Speed Battle Elapsed Time:", endSpeed - startSpeed)

Normal Battle Elapsed Time: 7.6664183139801025
Speed Battle Elapsed Time: 0.4333150386810303


In [11]:
strongest_id = normal_result.argmax()
strongest = pokeList[strongest_id]
print(strongest)

max_attack = max([p.attack for p in pokeList])

print("Enemy of Cloyster at leaste needs to be SuperEffective variety, because max_attack * 1.2 = ", max_attack * 1.2, "<", strongest.defense)

Monster(name='Cloyster', index=90, attack=95, defense=180, speed=85, hitpoint=[50, 50], variety=['Water', 'Ice'])
Enemy of Cloyster at leaste needs to be SuperEffective variety, because max_attack * 1.2 =  160.79999999999998 < 180


In [12]:
print("Pokemon Defense Ranking")
defense_ranking = np.argsort([p.defense for p in pokeList])[::-1]
for p in defense_ranking[:10]:
    print(pokeList[p])

Pokemon Defense Ranking
Monster(name='Cloyster', index=90, attack=95, defense=180, speed=85, hitpoint=[50, 50], variety=['Water', 'Ice'])
Monster(name='Onix', index=94, attack=45, defense=160, speed=30, hitpoint=[35, 35], variety=['Rock', 'Ground'])
Monster(name='Golem', index=75, attack=110, defense=130, speed=55, hitpoint=[80, 80], variety=['Rock', 'Ground', 'Electric'])
Monster(name='Omastar', index=138, attack=60, defense=125, speed=115, hitpoint=[70, 70], variety=['Rock', 'Water'])
Monster(name='Weezing', index=109, attack=90, defense=120, speed=85, hitpoint=[65, 65], variety=['Poison'])
Monster(name='Rhydon', index=111, attack=130, defense=120, speed=45, hitpoint=[105, 105], variety=['Ground', 'Rock'])
Monster(name='Graveler', index=74, attack=95, defense=115, speed=45, hitpoint=[55, 55], variety=['Rock', 'Ground', 'Electric'])
Monster(name='Kingler', index=98, attack=130, defense=115, speed=50, hitpoint=[55, 55], variety=['Water'])
Monster(name='Tangela', index=113, attack=55, d

In [13]:
print("Pokemon Attack Ranking")
attack_ranking = np.argsort([p.attack for p in pokeList])[::-1]
for p in attack_ranking[:10]:
    print(pokeList[p])

Pokemon Attack Ranking
Monster(name='Dragonite', index=148, attack=134, defense=95, speed=100, hitpoint=[91, 91], variety=['Dragon', 'Flying'])
Monster(name='Rhydon', index=111, attack=130, defense=120, speed=45, hitpoint=[105, 105], variety=['Ground', 'Rock'])
Monster(name='Kingler', index=98, attack=130, defense=115, speed=50, hitpoint=[55, 55], variety=['Water'])
Monster(name='Machamp', index=67, attack=130, defense=80, speed=65, hitpoint=[90, 90], variety=['Fighting'])
Monster(name='Flareon', index=135, attack=130, defense=60, speed=110, hitpoint=[65, 65], variety=['Fire'])
Monster(name='Pinsir', index=126, attack=125, defense=100, speed=55, hitpoint=[65, 65], variety=['Bug'])
Monster(name='Gyarados', index=129, attack=125, defense=79, speed=100, hitpoint=[95, 95], variety=['Water', 'Flying'])
Monster(name='Hitmonlee', index=105, attack=120, defense=53, speed=35, hitpoint=[50, 50], variety=['Fighting'])
Monster(name='Kabutops', index=140, attack=115, defense=105, speed=70, hitpoint

In [14]:
normal_result_id = normal_result.argsort()[::-1]
np.where(normal_result_id == 94)

(array([74]),)

In [15]:
round(180 * 0.7)

126

Effective pokemons should have attack > 126  
Or who have varieties against Water and Ice are...
- Grass
- Electric
- Fighting
- Rock
- Steel

and have enough attack

In [16]:
effective_varieties = ['Grass', 'Electric', 'Fighting', 'Rock', 'Steel']
effective_pokeList = [p for p in pokeList if np.any([v in p.variety for v in effective_varieties]) or p.attack > 126]
print("# of Pokemons who can fight Cloyster:", len(effective_pokeList), "/", len(pokeList))

winnable_pokeList = [p for p in effective_pokeList if p.attack * 1.3 * 1.2 > 180 * 0.7]
print("\n# of Pokemons who can give a damage to Cloyster:", len(winnable_pokeList), "/", len(pokeList))

for p in winnable_pokeList:
    print(p)

# of Pokemons who can fight Cloyster: 49 / 151

# of Pokemons who can give a damage to Cloyster: 24 / 151
Monster(name='Venusaur', index=2, attack=82, defense=83, speed=100, hitpoint=[80, 80], variety=['Grass', 'Poison'])
Monster(name='Raichu', index=25, attack=90, defense=55, speed=90, hitpoint=[60, 60], variety=['Electric', 'Psychic'])
Monster(name='Sandslash', index=27, attack=100, defense=110, speed=55, hitpoint=[75, 75], variety=['Ground', 'Steel'])
Monster(name='Parasect', index=46, attack=95, defense=80, speed=80, hitpoint=[60, 60], variety=['Bug', 'Grass'])
Monster(name='Primeape', index=56, attack=105, defense=60, speed=60, hitpoint=[65, 65], variety=['Fighting'])
Monster(name='Poliwrath', index=61, attack=85, defense=95, speed=70, hitpoint=[90, 90], variety=['Water', 'Fighting'])
Monster(name='Machoke', index=66, attack=100, defense=70, speed=50, hitpoint=[80, 80], variety=['Fighting'])
Monster(name='Machamp', index=67, attack=130, defense=80, speed=65, hitpoint=[90, 90], var

In [17]:
K = 1000
cloyster = pokedex["Cloyster"]
challengers = winnable_pokeList

win_rates = []
for challenger in challengers:
    beat = 0
    for i in range(K):
        result = pf.battle(cloyster, challenger)
        if result == 1: beat += 1
    win_rates.append([challenger, beat])
win_rates = np.array(win_rates)
    
ranking = win_rates[np.argsort(np.array(win_rates)[:, 1])][::-1]
for challenger, beat in ranking:
    print(f"Challenger {challenger.name} Win Rate: {beat}/{K}")

Challenger Rhydon Win Rate: 745/1000
Challenger Machamp Win Rate: 455/1000
Challenger Golem Win Rate: 117/1000
Challenger Dragonite Win Rate: 36/1000
Challenger Victreebel Win Rate: 2/1000
Challenger Hitmonlee Win Rate: 1/1000
Challenger Primeape Win Rate: 1/1000
Challenger Weepinbell Win Rate: 0/1000
Challenger Raichu Win Rate: 0/1000
Challenger Sandslash Win Rate: 0/1000
Challenger Parasect Win Rate: 0/1000
Challenger Poliwrath Win Rate: 0/1000
Challenger Machoke Win Rate: 0/1000
Challenger Aerodactyl Win Rate: 0/1000
Challenger Flareon Win Rate: 0/1000
Challenger Electabuzz Win Rate: 0/1000
Challenger Graveler Win Rate: 0/1000
Challenger Zapdos Win Rate: 0/1000
Challenger Kingler Win Rate: 0/1000
Challenger Exeggutor Win Rate: 0/1000
Challenger Hitmonchan Win Rate: 0/1000
Challenger Rhyhorn Win Rate: 0/1000
Challenger Kabutops Win Rate: 0/1000
Challenger Venusaur Win Rate: 0/1000


この結果からわかるように，パルシェンにはサイドンをぶつけるのが有効である．  
ちなみにSpeedBattleのようにパラメータで勝負させるとみんな負ける．

## ポケモンチーム戦
前提
- 1チームは6匹のポケモンで構成される
- 勝ったポケモンが次のポケモンと戦う
- 引き分けの場合，両者敗北とし，それぞれ次のポケモンを戦わせる
- 先に相手のポケモン全員に勝った方の勝ち

制約
- 同じポケモンを編成可能かどうか
- 相手のチームがわかっているか

In [18]:
def TeamBattle(team1, team2, k=1, battle=pf.battle, verbose=False):
    team_pokemon_num = 6
    
    # reset HP
    for p in range(team_pokemon_num):
        pf.ResetHP(team1[p])
        pf.ResetHP(team2[p])
    
    if verbose:
        print("\n======== Team Battle =======\n")
        print("Team1")
        for p in range(team_pokemon_num): print(team1[p])
        print()
        print("Team2")
        for p in range(team_pokemon_num): print(team2[p])
        print()
    
    team1_win = 0
    team2_win = 0
    
    for i in range(k):
        if verbose:
            print(f"\n======= Battle {i + 1} =======\n")
        
        # reset HP
        for p in range(team_pokemon_num):
            pf.ResetHP(team1[p])
            pf.ResetHP(team2[p])
        
        team1_remain = team_pokemon_num
        team2_remain = team_pokemon_num
        
        # Team Battle Start
        while team1_remain > 0 and team2_remain > 0:
            p1 = team1[-team1_remain]
            p2 = team2[-team2_remain]
            
            if verbose:
                print(f"{p1.name} vs {p2.name}")
                print(p1)
                print(p2)
            
            result = battle(p1, p2, resetHP=False)
            if result == 0: 
                team2_remain -= 1
                if verbose:
                    print(f"{p1.name} win")
            elif result == 1: 
                team1_remain -= 1
                if verbose:
                    print(f"{p2.name} win")
            else:
                team1_remain -=1
                team2_remain -=1
                if verbose:
                    print("Draw")
            
            if verbose:
                print(f"Team1 remain: {team1_remain}")
                print(f"Team2 remain: {team2_remain}")
                print()
        
        if team1_remain > team2_remain:
            team1_win += 1
            if verbose: print("Team1 win")
        elif team1_remain < team2_remain:
            team2_win += 1
            if verbose: print("Team2 win")
        else :
            if verbose: print("Draw")
    
    return (team1_win, team2_win)

In [19]:
team1 = [pokeList[p] for p in np.random.randint(0, len(pokeList),6)]
team2 = [pokeList[p] for p in np.random.randint(0, len(pokeList),6)]

TeamBattle(team1, team2, k=100, verbose=False)

(57, 43)

TeamBattleができるようになったところで最強チームを決めたい．  
とりあえず最強チームは総当たりランキング上位6匹にしてみる

In [20]:
strongest_team = [pokeList[p] for p in normal_result.argsort()[::-1][:6]]

こいつがどれだけ他のチームより強いか確かめたいが，チームの組み合わせは100兆を超えるので，総当たりは現実的でない．  
組み合わせでも100億=10^10

In [21]:
from itertools import permutations
all_teams = permutations(range(len(pokeList)), r=6)
151 ** 6

11853911588401

組み合わせだと100億=10^10  
これも現実的でない

In [24]:
from itertools import combinations
all_teams = combinations(range(len(pokeList)), r=1)
151*150*149*148*147*146 / (6*5*4*3*2)

14888600755.0